# Generate Reactions

This script performs the same task as the script in `scripts/generateReactions.py` but in visual ipynb format.
It can also evaluate the reaction forward and reverse rates at a user selected temperature.

In [ ]:
from rmgpy.rmg.main import RMG
from rmgpy.rmg.model import CoreEdgeReactionModel
from rmgpy import settings
from IPython.display import display
from arkane.output import prettify

Declare database variables here by changing the thermo and reaction libraries, or restrict to certain reaction families.  

In [ ]:
database = """
database(
    thermoLibraries = ['BurkeH2O2','primaryThermoLibrary','DFT_QCI_thermo','CBS_QB3_1dHR','Narayanaswamy','Chernov'],		
    reactionLibraries = [],
    seedMechanisms = [],
    kineticsDepositories = ['training'],
    kineticsFamilies = [
        'H_Abstraction',
        'R_Addition_MultipleBond',
        'intra_H_migration',
        'Intra_R_Add_Endocyclic',
        'Intra_R_Add_Exocyclic'
    ],
    kineticsEstimator = 'rate rules',
)

options(
    verboseComments=True,  # Set to True for detailed kinetics comments
)
"""

List all species you want reactions between

In [ ]:
species_list = """
species(
    label='i1',
    reactive=True,
    structure=adjacencyList(
        \"""
        multiplicity 2
        1 C u0 p0 c0 {3,S} {4,S} {10,S} {11,S}
        2 C u0 p0 c0 {4,S} {12,S} {13,S} {14,S}
        3 C u0 p0 c0 {1,S} {5,B} {6,B}
        4 C u1 p0 c0 {1,S} {2,S} {15,S}
        5 C u0 p0 c0 {3,B} {8,B} {19,S}
        6 C u0 p0 c0 {3,B} {9,B} {20,S}
        7 C u0 p0 c0 {8,B} {9,B} {17,S}
        8 C u0 p0 c0 {5,B} {7,B} {16,S}
        9 C u0 p0 c0 {6,B} {7,B} {18,S}
        10 H u0 p0 c0 {1,S}
        11 H u0 p0 c0 {1,S}
        12 H u0 p0 c0 {2,S}
        13 H u0 p0 c0 {2,S}
        14 H u0 p0 c0 {2,S}
        15 H u0 p0 c0 {4,S}
        16 H u0 p0 c0 {8,S}
        17 H u0 p0 c0 {7,S}
        18 H u0 p0 c0 {9,S}
        19 H u0 p0 c0 {5,S}
        20 H u0 p0 c0 {6,S}
        \"""
    )
)
"""

In [ ]:
# Write input file to disk
with open('temp/input.py','w') as input_file:
    input_file.write(database)
    input_file.write(species_list)

In [ ]:
# Execute generate reactions
from rmgpy.tools.generate_reactions import RMG, execute
kwargs = {
    'walltime': '00:00:00:00',
    'kineticsdatastore': True
}
rmg = RMG(input_file='temp/input.py', output_directory='temp')
rmg = execute(rmg, **kwargs)

In [ ]:
# Pick some temperature to evaluate the forward and reverse kinetics
T = 623.0 # K

In [ ]:
for rxn in rmg.reaction_model.output_reaction_list:
    print('=========================')
    display(rxn)
    print('Reaction Family = {0}'.format(rxn.family))
    print('')
    print('Reactants')
    for reactant in rxn.reactants:
        print('Label: {0}'.format(reactant.label))
        print('SMILES: {0}'.format(reactant.molecule[0].to_smiles()))
        print('')
    print('Products')
    for product in rxn.products:
        print('Label: {0}'.format(product.label))
        print('SMILES: {0}'.format(product.molecule[0].to_smiles()))
    print('')
    print(rxn.to_chemkin())
    print('')
    print('Heat of Reaction = {0:.2F} kcal/mol'.format(rxn.get_enthalpy_of_reaction(623.0)/4184))
    print('Forward kinetics at {0} K: {1:.2E}'.format(T, rxn.get_rate_coefficient(T)))

    reverseRate = rxn.generate_reverse_rate_coefficient()
    print('Reverse kinetics at {0} K: {1:.2E}'.format(T, reverseRate.get_rate_coefficient(T)))